In [44]:
# 원티드 채용공고 크롤링
import requests
import datetime
import json

def crawling_wanted_data(parsed_data, info_keys):
    result={}
    for key in info_keys:
        if key == 'company':   # 회사 이름
            result['company'] = parsed_data['job']['company']['name']  
        elif key == 'intro':   # 회사 소개
            result['intro'] = parsed_data['job']['detail']['intro'] 
        elif key == 'location':  # 근무 지역
            result['location'] = parsed_data['job']['address']['full_location'] 
        elif key == 'title':  # 제목
            result['title'] = parsed_data['job']['position']
        elif key == 'main_job': # 주요업무
            result['main_job'] = parsed_data['job']['detail']['main_tasks']
        elif key == 'require':  # 자격요건
            result['require'] = parsed_data['job']['detail']['requirements']
        elif key == 'prefer_point':  # 우대사항
            result['prefer_point'] = parsed_data['job']['detail']['preferred_points']
        elif key == 'benefits':  # 혜택 및 복지
            result['benefits'] = parsed_data['job']['detail']['benefits']
        elif key == 'deadline':  # 마감날짜 None = 상시
            result['deadline'] = parsed_data['job']['due_time']
        elif key == 'geo_location': # 근무지역 (위도,경도)
            geo_location = parsed_data['job']['address'].get('geo_location')
            if geo_location:
                result['lat'] = geo_location['n_location']['lat']
                result['lng'] = geo_location['n_location']['lng']
            else:
                result['lat'] = None
                result['lng'] = None
    return result

parsed_data = []

timestamp = int(datetime.datetime.now().timestamp())
tag_type_ids = 518
years = -1
limit = 10
offset = 0
# tag_type_ids: 518 개발/ 507 경영-비즈니스 /523 마케팅 /511 디자인 /530 영업 /510 고객서비스-리테일 /524 미디어 /513 엔지니어링-설계
# 517 HR /959 게임 제작 /508 금융 /522 제조-생산 /515 의료-제약-바이오 /10101 교육 /532 물류-무역 /10057 식-음료 /521 법률-법집행기관 /509 건설-시설 /514 공공-복지
# limit -> 100이면 100개씩 데이터추출 (한번에 100밖에 불가)
# offset -> 0번째부터 추출

# 10000개 데이터 가져오기 
total_data_count = 10

base_url = "https://www.wanted.co.kr/api/v4/jobs/"
for offset in range(0, total_data_count, limit):
    
    url = "https://www.wanted.co.kr/api/v4/jobs?{}&country=kr&tag_type_ids={}&job_sort=job.latest_order&locations=all&years={}&limit={}&offset={}".format(timestamp, tag_type_ids, years, limit, offset)
    parse_company = requests.get(url).json()
    
    recur_ids = [temp['id'] for temp in parse_company['data']]
    

    for recur_id in recur_ids:
        detail_url = f"{base_url}{recur_id}?{timestamp}"
        job_data = requests.get(detail_url).json()

        info_keys = ['company', 'title','intro', 'location', 'main_job','require', 'prefer_points', 'benefits','deadline', 'geo_location']  # 필요한 정보 키
        crawled_data = crawling_wanted_data(job_data, info_keys)
        
        print(json.dumps(crawled_data, indent=4, ensure_ascii=False))
        


{
    "company": "프렉스코리아(Presto)",
    "title": "Senior Mobile Engineer (Mobile Platform)",
    "intro": "* \"프렉스코리아\"는 Presto Labs의 거래소 프로덕트 개발 한국 법인 명칭입니다.\nPresto Labs는 2014년에 싱가포르에 설립된 알고리즘 트레이딩 전문 회사입니다. 우리는 전세계의 다양한 금융 상품을 계량적이고 자동화된 방법으로 거래하고 있습니다. Presto Labs는 사람이 투자 결정을 내릴 때 생길 수 있는 편향을 최소화하고, 거래를 보다 체계적으로 하기 위해 통계 분석 및 투자 결정을 자동화하는 데 중점을 둡니다.\n\n프렉스코리아는 팀원들이 창조적인 재능을 발휘할 수 있는 충분한 자유와 기회를 제공합니다. 이를 위해 우리는 자유로운 업무 분위기를 조성하는 한편, 수평적인 조직 문화를 구축해가고 있습니다. 또한 우리는 여러분들의 훌륭한 아이디어가 단지 제안 단계에서만 머물지 않고, 실제로 구현될 수 있도록 격려합니다. 프렉스코리아는 여러분들이 업무 외적인 것에 신경 쓰지 않고, 오직 업무에만 집중할 수 있는 환경을 제공합니다.\n\n프렉스코리아는 아직 해결되지 못한 문제를 풀어가는데 흥미를 느끼는 사람들이 모여있는 팀입니다. 학구적이고 열린 분위기에서 흥미로운 문제들을 함께 풀어나갈 재능 있는 분들을 적극적으로 찾고 있습니다.\n\n\nMobile Platform Team at Flipster\n\nFlipster에서 Mobile Platform Team은 아래 미션을 수행하기 위한 팀입니다.\n\n1. Engineering Productivity 와  Throughput 개선\n2. 성능과 같은 Non-functional 요구사항 관리 및 개선 프로젝트 리딩\n3. 모바일 플랫폼 표준화\n\n구체적으로는 다음과 같은 R&R을 가지고 Android, iOS, Backend, QA, Product Manager, Design, Data Enginee